In [3]:
import requests
import pandas as pd

In [1]:
url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"

In [16]:
url2 = "https://apps.fs.usda.gov/fiadb-api/fullreport?lat=40&lon=84&radius=0.5&rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"

In [17]:
# make request
resp = requests.get(url2)

In [18]:

# parse response to json
data = resp.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:


    # create output dictionary and populate it with estimate data frames
outDict = {}
    # append estimates
outDict['estimates'] = pd.DataFrame(data['estimates'])

    # append subtotals and totals if present
if 'subtotals' in data.keys():
    subT = {}
    for i in data['subtotals'].keys():
        subT[i] = pd.DataFrame(data['subtotals'][i])
    outDict['subtotals'] = subT
    outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
outDict['metadata'] = data['metadata']

In [5]:
outDict["estimates"]

,ESTIMATE,GRP1,GRP2,PLOT_COUNT,SE,SE_PERCENT,VARIANCE
0,338396.723126,`0001 Timberland,`0001 Timberland,126,14124.919150,4.174071,1.995133e+08
1,11687.686340,`0002 Reserved Forestland,`0003 Reserved productive forestland,6,5017.987809,42.933971,2.518020e+07
2,1807.524799,`0002 Reserved Forestland,`0004 Reserved other forestland,1,1923.492946,106.415854,3.699825e+06
3,1542.995199,`0003 Other forestland,`0002 Other forestland,1,1561.297876,101.186178,2.437651e+06
4,869624.874848,`0004 Nonforest,`0005 Nonforest,293,15297.520211,1.759094,2.340141e+08
5,11522.496930,`0005 Non-Census water,`0006 Non-Census water,7,5466.142491,47.438871,2.987871e+07
6,62249.798757,`0006 Census water,`0007 Census water,17,6553.197826,10.527259,4.294440e+07
